In [389]:
import os
import numpy as np 
import pandas as pd 
import matplotlib 
import matplotlib.pyplot as plt
import joblib

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, accuracy_score

from scipy.stats import randint

from pandas import DataFrame


from xgboost import XGBClassifier



# The Attributess include:
- Age: age of the patient [years]
- Sex: sex of the patient 
  * M: Male 
  * F: Female
- ChestPainType: chest pain type 
  * TA: Typical Angina 
  * ATA: Atypical Angina 
  * NAP: Non-Anginal Pain 
  * ASY: Asymptomatic
- RestingBP: resting blood pressure [mm Hg]
- Cholesterol: serum cholesterol [mm/dl]
- FastingBS: fasting blood sugar 
  * 1: if FastingBS > 120 mg/dl 
  * 0: otherwise
- RestingECG: resting electrocardiogram results 
  * Normal: Normal 
  * ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
  * LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria
- MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
- ExerciseAngina: exercise-induced angina 
  * Y: Yes 
  * N: No
- Oldpeak: oldpeak = ST [Numeric value measured in depression]
- ST_Slope: the slope of the peak exercise ST segment 
  * Up: upsloping 
  * Flat: flat, Down: downsloping
- HeartDisease: output class 
  * 1: heart disease 
  * 0: Normal

In [313]:
df=pd.read_csv("./data/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [314]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [315]:
df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [316]:
# No missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [317]:
# dataset is somewhat balanced
df["HeartDisease"].value_counts()

HeartDisease
1    508
0    410
Name: count, dtype: int64

In [318]:
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"].copy()

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

### Data transformations
* Since we'll be using tree based models, one-hot encoding is not necessary.
* Only transformation is labeling the categorical features (also handling outliers but this data set doesn't have any).
* Although the data does not contain any missing values, we'll add an imputer to our pipeline in case inference examples
  will have missing data.

In [320]:
def apply_label_encoding(X: DataFrame):
  transformed_df = DataFrame()
  for column_num in range(X.shape[1]):
      column = X[:, column_num]
      transformed_df[column_num] = LabelEncoder().fit_transform(column)

  return transformed_df

cat_pipeline = Pipeline([
  ("most_frequent", SimpleImputer(strategy="most_frequent")),
  ("label_encoder", FunctionTransformer(apply_label_encoding, feature_names_out="one-to-one"))
])

num_pipeline = Pipeline([
  ("impute", SimpleImputer(strategy="median")),
  ("standardize", StandardScaler())
])

num_columns = ["Age", "RestingBP", "Cholesterol", "FastingBS", "MaxHR", "Oldpeak"]
cat_columns = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

preprocessing = ColumnTransformer([
  ("num", num_pipeline, num_columns),
  ("cat", cat_pipeline, cat_columns)
])

X_train_transformed = preprocessing.fit_transform(X_train)

X_train_trans_df = DataFrame(X_train_transformed, 
                        columns=preprocessing.get_feature_names_out(),
                        index=X_train.index)

X_train_trans_df.head()

,num__Age,num__RestingBP,num__Cholesterol,num__FastingBS,num__MaxHR,num__Oldpeak,cat__Sex,cat__ChestPainType,cat__RestingECG,cat__ExerciseAngina,cat__ST_Slope
637,-1.148633,-0.975836,0.931322,-0.543754,1.721816,0.304911,1.0,0.0,1.0,0.0,1.0
541,2.328972,-1.578851,-0.826767,-0.543754,-0.623108,2.455047,1.0,2.0,0.0,0.0,0.0
570,0.221333,-0.263182,0.191074,-0.543754,-0.661549,1.052784,1.0,0.0,2.0,1.0,0.0
611,0.853625,0.120555,-0.586187,-0.543754,0.030396,-0.629931,1.0,3.0,2.0,0.0,2.0
685,0.748243,-0.701738,0.533439,-0.543754,0.145720,2.548531,1.0,0.0,1.0,1.0,1.0


In [322]:
def get_full_pipeline(clf):
  preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_columns),
    ("cat", cat_pipeline, cat_columns)
  ])
  
  return Pipeline([
    ("preprocessing", preprocessing),
    ("clf", clf)
  ])

def k_fold_cross_val_all_metrics(estimator):
  acc = cross_val_score(estimator, X_train, y_train, cv=5, scoring="accuracy")
  recall = cross_val_score(estimator, X_train, y_train, cv=5, scoring="recall")
  precision = cross_val_score(estimator, X_train, y_train, cv=5, scoring="precision")
  f1 = cross_val_score(estimator, X_train, y_train, cv=5, scoring="f1")

  print ("Accuracy:  ", np.mean(acc).round(2))
  print ("Recall:    ", np.mean(recall).round(2))
  print ("Precision: ", np.mean(precision).round(2))
  print ("F1:        ", np.mean(f1).round(2))

In [323]:
#Model 1: Random Forest
forest_clf = RandomForestClassifier(n_estimators=200,criterion="entropy")
forest_pipeline = get_full_pipeline(forest_clf)
k_fold_cross_val_all_metrics(forest_pipeline)

Accuracy:   0.85
Recall:     0.9
Precision:  0.85
F1:         0.86


In [324]:
# Model 2: XGBoost 
xgboost_clf = XGBClassifier(n_estimators=200)
xgboost_pipeline = get_full_pipeline(xgboost_clf)
k_fold_cross_val_all_metrics(xgboost_pipeline)


Accuracy:   0.84
Recall:     0.86
Precision:  0.85
F1:         0.86


In [383]:
# Training the forest based model on the entire training dataset (minus validation) and checking performance
clf = RandomForestClassifier(n_estimators=30, max_depth=100, criterion="entropy", random_state=42)
pipeline = get_full_pipeline(clf)

X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_train, y_train, stratify=y_train, random_state=42)
pipeline.fit(X_train_1, y_train_1)

predictions_train = pipeline.predict(X_train)
predictions_val = pipeline.predict(X_val_1)

print ("Accuracy val:  ", accuracy_score(y_val_1, predictions_val))
print ("Precision val: ", precision_score(y_val_1, predictions_val))
print ("Recall val:    ", recall_score(y_val_1, predictions_val))
print ("F1 val:        ", f1_score(y_val_1, predictions_val))
print ("----------")
print ("Accuracy train:  ", accuracy_score(y_train, predictions_train))
print ("Precision train: ", precision_score(y_train, predictions_train))
print ("Recall train:    ", recall_score(y_train, predictions_train))
print ("F1 train:        ", f1_score(y_train, predictions_train))


Accuracy val:   0.872093023255814
Precision val:  0.8762886597938144
Recall val:     0.8947368421052632
F1 val:         0.8854166666666666
----------
Accuracy train:   0.9680232558139535
Precision train:  0.9686684073107049
Recall train:     0.973753280839895
F1 train:         0.9712041884816753


In [413]:
# Randomized search

params_ditribs = {'clf__n_estimators': randint(low=5, high=50), 
                  'clf__max_depth': randint(low=10, high=100), 
                  'clf__criterion': ['gini', 'entropy', 'log_loss']}

clf_rnd = RandomForestClassifier(random_state=42)
pipeline_rnd = get_full_pipeline(clf_rnd)

scoring = 'f1'
rnd_search = RandomizedSearchCV(pipeline_rnd, param_distributions=params_ditribs, n_iter=10, cv=3,
                                scoring=scoring, random_state=42)

rnd_search.fit(X_train, y_train)

# Getting the best model and its hyperparameters
final_pipeline = rnd_search.best_estimator_
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="rank_test_score", ascending=True, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__criterion,param_clf__max_depth,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
2,0.033495,0.000510,0.006511,0.000146,log_loss,92,27,"{'clf__criterion': 'log_loss', 'clf__max_depth...",0.883019,0.881890,0.860377,0.875095,0.010417,1
0,0.043463,0.001963,0.007875,0.000272,log_loss,61,33,"{'clf__criterion': 'log_loss', 'clf__max_depth...",0.879699,0.881890,0.858209,0.873266,0.010684,2
3,0.034755,0.000557,0.006795,0.000196,log_loss,84,28,"{'clf__criterion': 'log_loss', 'clf__max_depth...",0.878788,0.876494,0.859316,0.871532,0.008689,3
1,0.032565,0.000433,0.006788,0.000246,log_loss,81,25,"{'clf__criterion': 'log_loss', 'clf__max_depth...",0.870229,0.877470,0.861423,0.869708,0.006562,4
8,0.060146,0.002399,0.011677,0.003913,entropy,31,48,"{'clf__criterion': 'entropy', 'clf__max_depth'...",0.866920,0.872000,0.865672,0.868197,0.002737,5


In [405]:
# Feature importance for selected model

preprocessing = final_pipeline["preprocessing"]
clf = final_pipeline["clf"]
clf.feature_importances_
preprocessing.get_feature_names_out()

features_with_importance = list(zip(preprocessing.get_feature_names_out(), clf.feature_importances_))
features_with_importance = sorted(features_with_importance, key=lambda tup: tup[1], reverse=True)
features_with_importance

[('cat__ST_Slope', 0.1671996415669509),
 ('cat__ChestPainType', 0.1442282770523757),
 ('num__Cholesterol', 0.1412140377662618),
 ('num__MaxHR', 0.11251546613024249),
 ('num__Age', 0.09699649416872314),
 ('num__Oldpeak', 0.09695456888675782),
 ('cat__ExerciseAngina', 0.08013085235000499),
 ('num__RestingBP', 0.07115174488429572),
 ('cat__Sex', 0.03241705477892192),
 ('cat__RestingECG', 0.031228425819598584),
 ('num__FastingBS', 0.02596343659586699)]

In [406]:
# Getting prediction by threshold

clf = final_pipeline["clf"]
pipeline.predict_proba(X_train) # prob: [FALSE, TRUE]


array([[0.53333333, 0.46666667],
       [0.2       , 0.8       ],
       [0.03333333, 0.96666667],
       ...,
       [0.03333333, 0.96666667],
       [0.13333333, 0.86666667],
       [0.9       , 0.1       ]])

In [414]:
# Final evaluation on the test set
predictions_test = final_pipeline.predict(X_test)

print ("Accuracy:   ", accuracy_score(y_test, predictions_test))
print ("Precision:  ", precision_score(y_test, predictions_test))
print ("Recall:     ", recall_score(y_test, predictions_test))
print ("F1:         ", f1_score(y_test, predictions_test))

Accuracy:    0.8913043478260869
Precision:   0.8923076923076924
Recall:      0.9133858267716536
F1:          0.9027237354085603


In [415]:
# Pickling the model
joblib.dump(final_pipeline, "heart_disease_model.pkl")

['heart_disease_model.pkl']